
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 2.2 LAB: Implementing Delta Sharing (Databricks-to-Databricks) \[Provider]

## Lab Overview
In this hands-on lab, you will implement Delta Sharing as a data provider. You'll work with a warehouse inventory management scenario where **TechWare Labs** (your organization) needs to share product and warehouse stock data with regional warehouse teams.

**Your Role:** Data Provider (TechWare Labs - Central Inventory Management)

**Scenario:**
You manage the central inventory system for TechWare Labs, a technology equipment distributor. Your regional warehouse managers need real-time access to product information and current stock levels to make informed decisions about reordering and distribution. Rather than creating data copies or building complex APIs, you'll use Delta Sharing to provide secure, live access to this data.

**Lab Objectives:**
By the end of this lab, you will be able to:
1. Inspect and understand the structure of data suitable for sharing
2. Create a Delta Share and add multiple tables to it
3. Configure a Databricks-to-Databricks recipient
4. Grant appropriate permissions to recipients
5. Verify and manage your shared data configuration

## Setup

Run the following cell to set up your lab environment. This will create:
- A catalog called `techware_labs`
- A schema called `inventory`
- Two tables: `products` and `warehouse_stock`

In [0]:
%run ./Includes/Lab-Setup-2

## Exercise 1: Inspect Your Data

Before sharing data, it's important to understand what you're working with. Let's explore the catalog, schema, and tables that have been created for you.

### Task 1.1: Check Your Current Context
**TODO:** Write a query to display your current catalog and schema.

In [0]:
SELECT current_catalog(), current_schema()


### Task 1.2: List Available Tables
**TODO:** Display all tables in the current schema.

In [0]:
SHOW TABLES;

### Task 1.3: Examine the Products Table
**TODO:** Query the `products` table to see what data it contains.

**Question:** How many products are in the table? What categories do they belong to?

In [0]:
SELECT * FROM products;

### Task 1.4: Examine the Warehouse Stock Table
**TODO:** Query the `warehouse_stock` table to understand inventory levels.

**Question:** How many warehouse locations do you see? Which products have stock in multiple warehouses?

In [0]:
SELECT * FROM warehouse_stock;

## Exercise 2: Create a Delta Share (15 minutes)

Now that you understand your data, you'll create a share to make it available to recipients. A share is a logical container for tables that you want to share.

### Task 2.1: Create the Share
**TODO:** Create a share named `warehouse_inventory` with an appropriate comment describing its purpose.

In [0]:
CREATE SHARE IF NOT EXISTS warehouse_inventory
COMMENT 'Share containing warehouse stock data for partner analysis';

### Task 2.2: Verify Share Creation
**TODO:** List all shares to confirm your share was created successfully.

In [0]:
SHOW SHARES;

## Exercise 3: Add Tables to the Share

With your share created, you now need to add the tables that recipients will be able to access.

### Task 3.1: Add the Products Table
**TODO:** Add the `products` table from `techware_labs.inventory` to the share with a descriptive comment.

In [0]:
ALTER SHARE warehouse_inventory 
ADD TABLE techware_labs.inventory.products
COMMENT 'Product master data';

### Task 3.2: Add the Warehouse Stock Table
**TODO:** Add the `warehouse_stock` table to the share with an appropriate comment.

In [0]:

ALTER SHARE warehouse_inventory 
ADD TABLE techware_labs.inventory.warehouse_stock
COMMENT 'Warehouse stock data';

### Task 3.3: Verify Tables in the Share
**TODO:** Display all assets (tables) that are currently in your share.

In [0]:
SHOW ALL IN SHARE warehouse_inventory;

## Exercise 4: Create and Configure a Recipient

Now you'll create a recipient configuration for your regional warehouse team. In a Databricks-to-Databricks sharing scenario, the recipient must provide their metastore ID.

### Background: Getting the Recipient's Metastore ID
In a real-world scenario, the recipient would provide their metastore ID to you. For this lab, you'll need to coordinate with your recipient workspace to obtain their metastore ID.

**The recipient can obtain their metastore ID by:**
1. Going to Catalog Explorer → Delta Sharing (gear icon)
2. Clicking "Copy Sharing Identifier"
3. Providing this identifier to you (the provider)

The Metastore ID can also be obtained using `system.information_schema.metastores` or `DESCRIBE METASTORE` __at the Recipient__.

### Task 4.1: Create the Recipient
**TODO:** Create a recipient named `regional_warehouse` using the metastore ID provided by your recipient workspace.

**Note:** Replace `'<recipient_metastore_id>'` with the actual metastore ID from your recipient workspace. The format should be similar to: `'aws:us-west-2:adc48801-0d35-4e94-9cbc-0ff7f29ce611'`

In [0]:
CREATE RECIPIENT IF NOT EXISTS regional_warehouse
USING ID '<your-recipient-metadata-id-here>'
COMMENT 'Recipient for D2D Delta Sharing lab';

### Task 4.2: Verify Recipient Creation
**TODO:** List all recipients to confirm your recipient was created.

In [0]:
SHOW RECIPIENTS;

### Task 4.3: Get Recipient Details
**TODO:** Describe the recipient to view its configuration details.

In [0]:
DESCRIBE RECIPIENT regional_warehouse;

## Exercise 5: Grant Access to the Recipient

The final step in the provider configuration is to grant the recipient access to your share. This establishes the sharing relationship.

### Task 5.1: Grant SELECT Permissions
**TODO:** Grant the recipient SELECT permissions on the share.

In [0]:
GRANT SELECT ON SHARE warehouse_inventory TO RECIPIENT regional_warehouse;

### Task 5.2: Verify Grants on the Share
**TODO:** Display all grants on the share to confirm the recipient has access.

In [0]:
SHOW GRANTS ON SHARE warehouse_inventory;

### Task 5.3: Verify Shares Available to Recipient
**TODO:** Check which shares the recipient has access to.

In [0]:
SHOW GRANTS TO RECIPIENT regional_warehouse;

## Switch to the Recipient Workspace 
Now switch over to the Recipient workspace to Mount the Share in a Catalog and query the share

## Exercise 6: Monitor Share Activity

After the recipient begins accessing your shared data, you can monitor their activity using the system tables.

### Task 6.1: Query Share Access Logs
**TODO:** Write a query to view credential vending activity for your share.

**Note:** This query will only return results after the recipient has accessed the shared data. Run this after completing the recipient lab.

In [0]:
SELECT 
event_time,
request_params.table_full_name,
request_params.recipient_name,
request_params.share_name
FROM system.access.audit 
WHERE action_name = 'generateTemporaryTableCredential'
AND user_agent = 'Delta Sharing'
ORDER BY event_time DESC;

## Summary and Next Steps

Congratulations! You've successfully:
✅ Created a Delta Share for your warehouse inventory  
✅ Added product and warehouse stock tables to the share  
✅ Configured a Databricks-to-Databricks recipient  
✅ Granted appropriate permissions to the recipient  
✅ Learned how to monitor share activity  

**Key Takeaways:**
- Delta Sharing enables secure data sharing without copying data
- Providers maintain full control over shared data
- Recipients get real-time access to the latest data
- All access is logged and auditable

**Remember:** In a production environment, you would typically:
- Use more restrictive permissions and row/column-level security
- Implement data governance policies
- Set up alerting for unusual access patterns
- Document sharing agreements with recipients

---
&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>